# **Designing, Coordinating, and Refining Collaborative AI Systems with ADK & Gemini**

## 📋 Project Overview

This project explores advanced agentic AI architectures built with Google’s Agent Development Kit (ADK) and Gemini models.
It demonstrates how to design, coordinate, and optimize teams of agents that work together — sequentially, in parallel, or iteratively — to complete complex reasoning and decision-making tasks.

Unlike the single-agent reasoning system developed in the earlier project, this notebook focuses on multi-agent orchestration — showing how specialized agents (Planner, Researcher, Critic, Refiner, etc.) can collaborate within structured workflows to achieve higher reliability, creativity, and control.

The implementation follows three core architecture patterns—Sequential, Parallel, and Loop—each mirroring real-world production scenarios where autonomous systems must divide labor, manage dependencies, and improve outputs over time.

## 🎯 Objectives


- Architectural Thinking: Knowing when and why to use multi-agent setups instead of a single model.

- Workflow Design: Implementing three essential orchestration patterns — Sequential, Parallel, and Loop.

- System Robustness: Building production-grade agents with retry logic, fault tolerance, and traceable communication.

- Practical Application: Translating architectural principles into real AI pipelines such as content generation, research analysis, and iterative refinement.

## 📚 Table of Contents

1. **Setup & Dependencies**

- Install and authenticate ADK & Gemini
- Import orchestration utilities
- Configure robust retry policies

2. **Foundation — Understanding Multi-Agent Systems**

- Why single agents plateau
- Benefits of specialization & coordination
- Core principles of multi-agent orchestration

3. **Sequential Workflow — The Assembly Line Pattern**

- Deterministic task ordering
- Building a blog-generation pipeline
- Implementing SequentialAgent with ADK

4. **Parallel Workflow — Independent Execution**

- Concurrent sub-tasks for efficiency
- Research & analysis pipelines
- Implementing ParallelAgent with ADK

5. **Loop Workflow — Iterative Refinement**

- Critique-and-refine cycles for quality control
- Evaluator → Refiner → Evaluator loops
- Implementing LoopAgent for adaptive optimization

6. **Summary & Decision Guide**

- Choosing the right pattern for a given use-case
- Comparative trade-offs (speed vs control vs quality)
- Best practices & common pitfalls

## 💡 Project Significance

This notebook highlights advanced AI Engineering capabilities — specifically, how to move from reasoning agents to autonomous multi-agent systems capable of collaboration and self-improvement.

It builds directly on my previous work,
👉 [Building a Reasoning AI Agent with ADK & Gemini](http://www.kaggle.com/code/kanikaw/building-a-reasoning-ai-agent-with-adk-gemini),
extending that foundation into a fully-architected system that mirrors real-world agentic AI infrastructure.

## **Section 1: Setup & Dependencies**

Before implementing multi-agent architectures, it’s essential to configure a reliable development environment.
This section establishes the foundational components for building, coordinating, and testing agentic AI systems using **Google’s Agent Development Kit (ADK)**.

### **1.1 — Install Dependencies**

**🧠 Conceptual Insight**
The **Agent Development Kit (ADK)** is Google’s production-grade framework for building autonomous and collaborative AI agents.
Unlike raw LLM API access, ADK abstracts orchestration, memory, and tool-calling layers into composable components, enabling developers to scale from single-agent prototypes to coordinated multi-agent systems.

**🔧 Implementation Notes**
Kaggle Notebooks already include the pre-installed **`google-adk`** library and its dependencies, so no additional installation steps are required for this environment.

For external or local development, ADK can be installed via PyPI:

```bash
pip install google-adk
```

**🔍 Engineering Rationale**
Dependency isolation and version control are critical for **reproducibility and system reliability**—especially in multi-agent ecosystems where orchestration logic and model versions must remain synchronized.
Maintaining a consistent ADK environment ensures that agent behaviors, coordination protocols, and evaluation metrics remain deterministic across experiments and deployments.

### **1.2 — Configure Gemini API Authentication**

To enable the agentic workflows in this notebook, we need to authenticate access to **Google’s Gemini API** — the model foundation for all reasoning and orchestration tasks.

**🔐 Purpose & Context**
The **Gemini API** powers the intelligence layer of each agent.
Proper authentication ensures that all API calls are secure, rate-limited, and compliant with Google Cloud’s authorization protocols.
This setup step mirrors standard practices used in production environments where credentials are stored securely and never hard-coded.

**🧩 Implementation Workflow**

1. **Generate an API Key**

   * Visit the [Google AI Studio → API Keys](https://aistudio.google.com/app/api-keys) page.
   * Create a new key or reuse an existing one tied to your Gemini project.

2. **Store the Key Securely in Kaggle**

   * In the top bar of the notebook, navigate to **`Add-ons → Secrets`**.
   * Create a new secret named **`GOOGLE_API_KEY`**.
   * Paste the key value and click **Save**.
   * Ensure the checkbox next to `GOOGLE_API_KEY` is selected so the secret attaches to this notebook session.

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### **1.3 — Import Core ADK Components**

To construct modular, collaborative AI systems, we begin by importing key building blocks from **Google’s Agent Development Kit (ADK)**.
These imports define the foundational abstractions for creating, coordinating, and testing agentic workflows.

### **🧠 Conceptual Insight**

Clean and explicit imports are essential for maintaining **readability, traceability, and modularity** within agent architectures.
Each component contributes a well-defined role in the orchestration stack — from the reasoning layer (Gemini) to execution environments (runners) and tool interfaces.

### **🔧 Implementation Summary**

The following modules are used throughout this notebook:

* **`Agent, SequentialAgent, ParallelAgent, LoopAgent`** —
  Define how agents operate individually or collaboratively (sequentially, concurrently, or iteratively).

* **`Gemini`** —
  The LLM interface that powers each agent’s reasoning and communication capabilities.

* **`InMemoryRunner`** —
  Provides a lightweight, local execution context for testing multi-agent interactions before production deployment.

* **`AgentTool, FunctionTool, google_search`** —
  Extend agent functionality through callable tools, enabling real-world interactions such as live search and computation.

* **`types`** —
  Contains configuration objects (e.g., retry policies, HTTP options) that support resilient system behavior.

In [2]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### 1.4: Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

## Section 2: Why Multi-Agent Systems?

**The Problem: The "Do-It-All" Agent**

While large language models are remarkably capable, a single monolithic agent quickly becomes inefficient when handling complex, multi-step tasks.
An agent attempting to research, analyze, write, and fact-check simultaneously often struggles with:

* Prompt bloat – oversized instruction context that dilutes clarity
* Debugging difficulty – unclear which reasoning step failed
* Inconsistency – outputs vary as task scope increases
* Maintenance overhead – every workflow change requires prompt rewrites

In short, the “do-it-all” design leads to fragile orchestration and limits scalability.

**The Solution: A Team of Specialists**

Instead of overloading a single agent, we distribute work across a multi-agent system — a coordinated team of specialized agents, each focused on a narrow, well-defined function.

For example:

* Research Agent → Retrieves and structures factual context
* Writer Agent → Synthesizes text based on structured inputs
* Editor or Critic Agent → Evaluates and refines the draft for clarity and accuracy

This division of responsibility mirrors real-world collaboration, improving modularity, transparency, and reliability. Each agent remains simple, testable, and interchangeable — a key advantage for scalable AI architectures.

**Architecture: Single Agent vs Multi-Agent Team**

The diagram below illustrates the shift from a single overloaded agent to a coordinated multi-agent loop that shares information via a central controller or shared state.

<!--
```mermaid
graph TD
    subgraph Single["❌ Monolithic Agent"]
        A["One Agent Does Everything"]
    end

    subgraph Multi["✅ Multi-Agent Team"]
        B["Root Coordinator"] -- > C["Research Specialist"]
        B -- > E["Summary Specialist"]

        C -- >|findings| F["Shared State"]
        E -- >|summary| F
    end

    style A fill:#ffcccc
    style B fill:#ccffcc
    style F fill:#ffffcc
```
-->

<img width="800" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/multi-agent-team.png" alt="Multi-agent Team" />

### 2.1 Example: Research & Summarization System

Let's build a system with two specialized agents:

1. **Research Agent** - Searches for information using Google Search
2. **Summarizer Agent** - Creates concise summaries from research findings

In [5]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",  # The result of this agent will be stored in the session state with this key.
)

print("✅ research_agent created.")

✅ research_agent created.


In [6]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The instruction is modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


Then we bring the agents together under a root agent, or coordinator:

In [7]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This instruction tells the root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


### **Integrating Sub-Agents as Tools**

In this step, we encapsulate each **specialized sub-agent** (e.g., Researcher, Writer, Critic) using the `AgentTool` abstraction.
This design pattern converts agents into **callable tools** that the **root coordinator agent** can dynamically invoke as needed — similar to how software modules expose APIs for task delegation.

By wrapping sub-agents as tools:

* The **root agent** orchestrates when and how each specialist contributes.
* **Inter-agent communication** becomes structured, traceable, and modular.
* The system maintains **clear boundaries** between reasoning, execution, and evaluation.

This approach mirrors real-world **multi-agent orchestration** pipelines, where a central controller routes tasks among autonomous workers based on context and intermediate results.


In [8]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


ResearchCoordinator > The latest advancements in quantum computing are poised to revolutionize AI by providing exponentially greater processing power. This will enable AI to tackle previously intractable problems, leading to enhanced data processing, faster learning, and improved accuracy in machine learning. Quantum algorithms can solve complex optimization tasks crucial for fields like logistics and drug discovery, and enhance autonomous systems and natural language processing. Significant progress is being made in quantum error correction, a key challenge for building reliable quantum computers. While large-scale quantum AI applications are still developing, a near-term approach involves hybrid quantum-classical computing. Major tech companies are investing heavily in this area, and the global market for quantum AI applications is expected to grow substantially.


## Section 3: Sequential Workflows - The Assembly Line

**The Problem: Uncontrolled Execution Order**

In the previous setup, multiple agents collaborated through free-form prompts and contextual instructions. While flexible, this approach can be unreliable — a large language model may skip steps, reorder actions, or over-optimize a response path.
When workflows depend on precise, ordered task execution, prompt-based sequencing alone is insufficient.

**The Solution: A Deterministic Pipeline**

To enforce predictable execution order, we introduce the SequentialAgent, which functions like an assembly line.
Each sub-agent performs a well-defined operation, and its output automatically feeds into the next stage. This ensures that:

- Tasks execute in a strict, linear sequence.
- Each step builds directly on validated output from the previous one.
- The pipeline remains auditable and debuggable at every hand-off.

This deterministic behavior is crucial in workflows such as document generation, report validation, or data-to-text pipelines — where missing or re-ordered steps could lead to incoherent results.

**When to Use Sequential Patterns**

Use a SequentialAgent when:

- Task order matters (e.g., outline → write → edit).
- Each step depends on prior context.
- You require traceable, reproducible output across runs.

This mirrors real-world data-processing pipelines — each agent is a node in a controlled flow, contributing a single, dependable transformation.

**Architecture: Blog Post Creation Pipeline**

<!--
```mermaid
graph LR
    A["User Input: Blog about AI"] -- > B["Outline Agent"]
    B -- >|blog_outline| C["Writer Agent"]
    C -- >|blog_draft| D["Editor Agent"]
    D -- >|final_blog| E["Output"]

    style B fill:#ffcccc
    style C fill:#ccffcc
    style D fill:#ccccff
```
-->

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/sequential-agent.png" alt="Sequential Agent" />

### 3.1 Example: Blog Post Creation with Sequential Agents

**Objective**
Demonstrate a deterministic, end-to-end content pipeline using a `SequentialAgent` that transforms a topic into a publish-ready draft through three specialized stages.

**Specialized Roles**

1. **Outline Agent** — Produces a structured outline (sections, subpoints, estimated word counts).
2. **Writer Agent** — Expands the outline into a cohesive first draft (intro, body, conclusion, transitions).
3. **Editor Agent** — Refines clarity, tone, and structure; enforces style rules; runs a light factual consistency pass.

**Orchestration (Assembly Line)**

* The **output** of the Outline Agent becomes the **input** to the Writer Agent.
* The Writer’s draft is passed to the **Editor Agent** for final polish.
* The `SequentialAgent` guarantees ordering and makes each hand-off **traceable**.

**Design Notes**

* **Contracted I/O:** Each agent accepts/returns well-defined fields (e.g., `{topic}`, `{outline}`, `{draft}`) to reduce prompt drift.
* **Guardrails:** Minimal style constraints (tone, target audience, length budget) travel with the payload rather than being re-prompted from scratch.
* **Recoverability:** Failures are isolated per stage; the sequence can retry a single step without rerunning the entire pipeline.

**Evaluation Signals** *(what to look for in the run)*

* Outline quality: coverage, logical flow, section balance.
* Draft quality: coherence, specificity, adherence to outline.
* Edit quality: readability, concision, consistency with brief.
* Determinism: identical input yields stable stage outputs across runs (given fixed seeds/configs).

**Acceptance Criteria**

* Produces a final draft with clear sections, consistent voice, and no dangling bullets/placeholders.
* Adheres to length/tone constraints defined in the pipeline context.
* Emits a **single** final artifact plus a short metadata block (e.g., word count, readability grade).

**Example Invocation (conceptual)**

* **Input:** `topic="How agentic AI changes product development workflows"`
* **Pipeline:** `Outline → Writer → Editor`
* **Outcome:** A polished blog draft ready for review, with stage artifacts logged for auditability.


In [9]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",  # The result of this agent will be stored in the session state with this key.
)

print("✅ outline_agent created.")

✅ outline_agent created.


In [10]:
# Writer Agent: Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # The `{blog_outline}` placeholder automatically injects the state value from the previous agent's output.
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",  # The result of this agent will be stored with this key.
)

print("✅ writer_agent created.")

✅ writer_agent created.


In [11]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # This agent receives the `{blog_draft}` from the writer agent's output.
    instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",
    output_key="final_blog",  # This is the final output of the entire pipeline.
)

print("✅ editor_agent created.")

✅ editor_agent created.


Then we bring the agents together under a sequential agent, which runs the agents in the order that they are listed:

In [12]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


Let's run the agent and give it a topic to write a blog post about:

In [13]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > ## Outline:

**Catchy Headline:** Level Up Your Code: Why Multi-Agent Systems are Your Next Big Developer Superpower

**Introduction Hook:** Tired of monolithic codebases that feel like a tangled ball of yarn? What if you could break down complex problems into smaller, smarter, and more collaborative pieces? Enter Multi-Agent Systems (MAS) – a powerful paradigm that's transforming how we build software, offering unprecedented flexibility, scalability, and problem-solving capabilities.

**Main Sections:**

**1. Deconstructing Complexity: The Power of Specialization**

*   **Divide and Conquer:** MAS allows you to break down large, intricate problems into smaller, manageable sub-problems, assigning each to a specialized "agent."
*   **Focused Development:** Developers can focus on building and optimizing individual agents with specific fun

---
## 🛣️ Section 4: Parallel Workflows - Independent Researchers

**The Problem: Sequential Bottleneck**

Sequential pipelines guarantee order and determinism, but they can become inefficient when handling independent subtasks.
For example, if an AI system needs to research three unrelated topics — technology, healthcare, and finance — running them sequentially forces each step to wait for the previous one, even though there’s no dependency between them.
This creates latency bottlenecks and underutilizes available computational resources.

**The Solution: Concurrent Execution with ParallelAgent**

When multiple tasks can be executed independently, the optimal solution is parallel orchestration.
The ParallelAgent enables concurrent execution of multiple sub-agents — each focusing on its own specialized domain — and then aggregates their results once all tasks complete.

This approach:

- Maximizes throughput by eliminating idle waiting time.
- Preserves task isolation so that one agent’s delay or failure doesn’t block others.
- Allows for scalable performance gains in both latency-sensitive and distributed systems.

**When to Use Parallel Patterns**

Adopt a ParallelAgent when:

- Tasks are independent (no shared state or dependencies).
- Speed and resource utilization are critical.
- You plan to aggregate results after simultaneous execution.

Typical use cases include:

- Multi-topic research
- Multi-document summarization
- Batch data labeling
- Ensemble decision-making

**Architecture: Multi-Topic Research**

The diagram below illustrates a research system where three domain-specific agents operate concurrently, with an aggregator agent compiling their findings into a unified report.

<!--
```mermaid
graph TD
    A["User Request: Research 3 topics"] -- > B["Parallel Execution"]
    B -- > C["Tech Researcher"]
    B -- > D["Health Researcher"]
    B -- > E["Finance Researcher"]

    C -- > F["Aggregator"]
    D -- > F
    E -- > F
    F -- > G["Combined Report"]

    style B fill:#ffffcc
    style F fill:#ffccff
```
-->

<img width="600" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png" alt="Parallel Agent" />

### 4.1 Example: Parallel Multi-Topic Research

This example demonstrates how to design a multi-agent research system where multiple specialized agents operate concurrently, each focusing on a different domain.
A final Aggregator Agent then synthesizes the collective insights into a single executive summary.
This architecture models real-world use cases like market research dashboards, multi-domain reporting, or ensemble reasoning.

1. **Tech Researcher** - Researches AI/ML news and trends
2. **Health Researcher** - Researches recent medical news and trends
3. **Finance Researcher** - Researches finance and fintech news and trends
4. **Aggregator Agent** - Combines all research findings into a single summary

**System Design**

**Objective:**
Execute three independent research agents (Technology, Health, Finance) in parallel and then combine their findings into a unified summary.

**Architecture:**
**1️⃣ Parallel Phase:**

- Tech, Health, and Finance researchers operate concurrently.
- Each writes to a shared state key (tech_research, health_research, finance_research).
- 
**2️⃣ Aggregation Phase:**

Once all research agents finish, the AggregatorAgent compiles the findings into an executive summary.

**3️⃣ Top-Level Control:**

A SequentialAgent orchestrates the two phases: Parallel research → Aggregation.

**Step 1: Define Specialized Research Agents**

Each sub-agent focuses on a single domain and uses the google_search tool to retrieve real-time insights.

In [14]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [15]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [16]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


**Step 2: Define the Aggregator Agent**

This agent runs after all parallel research agents have completed.
It reads their outputs from shared memory, merges them, and generates a cohesive executive summary.

In [17]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


**Step 3: Build the Parallel and Sequential Workflow**

Here, we encapsulate the three research agents within a ParallelAgent,
then wrap that inside a SequentialAgent to enforce the two-stage execution order.

In [18]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


Let's run the agent and give it a prompt to research the given topics:

In [19]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
FinanceResearcher > Here's your executive briefing for November 17, 2025:

**Technology:** Cloud computing is increasingly central to business strategy, with AI becoming its core. Hybrid and multi-cloud approaches are normalizing for flexibility and risk mitigation. Quantum computing's accessibility via cloud platforms is expected to drive innovation in fields like drug discovery.

**Health:** AI is revolutionizing healthcare, projected for explosive market growth. Key benefits include enhanced clinical accuracy, efficiency, and personalized patient care. However, challenges like data bias and security require careful management.

**Finance:** Fintech is rapidly evolving, driven by generative AI for hyper-personalization of services and enhanced customer support. Open banking continues to expand, while decentralized finance (DeFi) is poised for a comeback, presenting new op

## ➰ Section 5: Loop Workflows - The Refinement Cycle

**The Problem: One-Shot Quality**

Traditional workflows like SequentialAgent and ParallelAgent are linear pipelines — they execute once and stop.
While this is efficient for deterministic tasks, it’s insufficient for problems that demand iterative improvement, such as writing, code generation, or reasoning verification.

If the first draft or output doesn’t meet quality expectations, the system has no built-in mechanism to self-review, critique, or refine. The result: static output with no learning loop.

**The Solution: Iterative Refinement with LoopAgent**

The LoopAgent introduces a feedback-based improvement cycle.
It repeatedly executes a set of sub-agents until a termination condition (such as “quality approved” or “max iterations reached”) is satisfied.

This pattern transforms the workflow into a self-improving system, enabling agents to:

- Generate → Evaluate → Revise → Repeat.
- Enhance quality with each iteration.
- Apply structured self-critique for higher accuracy and polish.

**When to Use Loop Architectures**

Use a LoopAgent when:

- The task benefits from iterative feedback (e.g., writing, design, reasoning chains).
- Quality control and self-correction are more valuable than single-pass speed.
- You want adaptive improvement through a review–refine cycle.

Common use cases include:

- Story or article refinement
- Code review and bug correction loops
- Response grading and regeneration
- Creative ideation or brainstorming sessions

**Architecture: Story Writing & Critique Loop**

This architecture models a writer–critic dynamic, where the system alternates between generating content and reviewing it.
The LoopAgent manages repeated exchanges until the output reaches acceptable quality or the iteration limit is reached.

<!--
```mermaid
graph TD
    A["Initial Prompt"] -- > B["Writer Agent"]
    B -- >|story| C["Critic Agent"]
    C -- >|critique| D{"Iteration < Max<br>AND<br>Not Approved?"}
    D -- >|Yes| B
    D -- >|No| E["Final Story"]

    style B fill:#ccffcc
    style C fill:#ffcccc
    style D fill:#ffffcc
```
-->

<img width="250" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/loop-agent.png" alt="Loop Agent" />

### 5.1 Example:Iterative Story Refinement (Writer ↔ Critic Loop)

**Objective**

Demonstrate a self-improving generation loop where a Writer drafts a short story and a Critic evaluates it, feeding structured feedback back to the Writer until quality criteria are met or a max-iteration cap is reached.

**Loop Pattern**
Writer → Critic → (approve? else re-write) → … → Final

**Termination**
- Primary: approved == true (Critic signals that the story meets criteria)
- Fallback: max_iterations reached (prevents infinite loops)

**Evaluation Signals**

- Narrative coherence, style adherence, constraint compliance (genre, word budget), and originality
- Critic produces actionable feedback (not generic praise)

The InitialWriterAgent is executed once at the start of the refinement cycle.
It generates the baseline story draft that will serve as the first input to the iterative critique loop.

- Role: Acts as the entry point for the LoopAgent pipeline.
- Function: Converts the user prompt into a coherent first draft, constrained to ~100–150 words.
- Output Key: current_story — this key stores the draft in shared session state so subsequent agents (Critic, Refiner) can access and update it.
- Design Rationale: Separating the initial generator from the refinement loop makes the system modular — you can swap or retrain the Writer component independently without altering the loop logic.

In [20]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
    output_key="current_story",  # Stores the first draft in the state.
)

print("✅ initial_writer_agent created.")

✅ initial_writer_agent created.


The CriticAgent serves as the evaluation and feedback component in the refinement loop.
It reviews the Writer’s current draft, scores it qualitatively, and either approves the story or generates targeted improvement suggestions.

- Role: Acts as a quality gate in the iteration cycle.
- Function: Reviews the story stored in {current_story}, evaluating structure, pacing, and completeness.
- Decision Logic:
  * If the story meets quality criteria → returns "APPROVED" (this halts the loop).
  * If not → returns 2–3 concrete, actionable suggestions for revision.
- Output Key: critique — the feedback or approval signal is written to shared state for the next iteration.
- Design Rationale: Keeping the critic tool-free enforces objective evaluation based solely on textual reasoning, minimizing noise from external data sources. This modular design lets you swap out the Critic for different evaluation policies (e.g., style-based, factuality-based, or sentiment-based) without changing the loop architecture.

In [24]:
# This agent's only job is to provide feedback or the approval signal. It has no tools.
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}
    
    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",  # Stores the feedback in the state.
)

print("✅ critic_agent created.")

✅ critic_agent created.


We separate “deciding the story is done” from “stopping the loop”:

- exit_loop (FunctionTool): a minimal tool that writes a structured flag into shared state (e.g., loop_exit=True), plus a reason.

- ApprovalWatcher (Agent): inspects {critique}; if it’s exactly APPROVED, it calls exit_loop(...). Otherwise, it emits CONTINUE.

This keeps the loop controller simple, auditable, and easy to test.

In [21]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}


print("✅ exit_loop function created.")

✅ exit_loop function created.


We expose a minimal exit_loop function as a FunctionTool so an agent can explicitly stop the loop.
Then we create a RefinerDeciderAgent that reads {critique} and either:

1. Calls exit_loop when the critic’s verdict is strictly APPROVED, or

2. Rewrites {current_story} using the critic’s actionable feedback.

This consolidates control logic into one “brain” for the loop, reducing hops and making behavior auditable.

In [22]:
# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a story refiner. You have a story draft and critique.
    
    Story Draft: {current_story}
    Critique: {critique}
    
    Your task is to analyze the critique.
    - IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.""",
    output_key="current_story",  # It overwrites the story with the new, refined version.
    tools=[
        FunctionTool(exit_loop)
    ],  # The tool is now correctly initialized with the function reference.
)

print("✅ refiner_agent created.")

✅ refiner_agent created.


Then we bring the agents together under a loop agent, which is itself nested inside of a sequential agent.

This design ensures that the system first produces an initial story draft, then the refinement loop runs up to the specified number of `max_iterations`:

In [25]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


Let's run the agent and give it a topic to write a short story about:

In [26]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias trimmed the lamp, the familiar scent of whale oil clinging to the air. For thirty years, the rhythm of the sea and the creak of the tower had been his world. Tonight, however, was different. A storm had battered the coast, and amongst the seaweed and driftwood washed ashore was a rolled-up parchment.

He unfurled it, his weathered fingers trembling. It wasn't paper, but a strange, leathery material. As he held it closer, intricate lines began to glow with an otherworldly blue light. They depicted not islands or coastlines he knew, but swirling patterns and celestial bodies. A single, pulsing star marked a spot in the vast, inky blackness. Elias, alone in his beacon, felt the immensity of the ocean shrink. This map was for somewhere far beyond the horizon.
CriticAgent > The story is a good start, setting a strong atmosphere 

--- 
## Section 6: Summary - Choosing the Right Pattern

### Decision Tree: Which Workflow Pattern?

<!--
```mermaid
graph TD
    A{"What kind of workflow do you need?"} -- > B["Fixed Pipeline<br>(A → B → C)"];
    A -- > C["Concurrent Tasks<br>(Run A, B, C all at once)"];
    A -- > D["Iterative Refinement<br>(A ⇆ B)"];
    A -- > E["Dynamic Decisions<br>(Let the LLM decide what to do)"];

    B -- > B_S["Use <b>SequentialAgent</b>"];
    C -- > C_S["Use <b>ParallelAgent</b>"];
    D -- > D_S["Use <b>LoopAgent</b>"];
    E -- > E_S["Use <b>LLM Orchestrator</b><br>(Agent with other agents as tools)"];

    style B_S fill:#f9f,stroke:#333,stroke-width:2px
    style C_S fill:#ccf,stroke:#333,stroke-width:2px
    style D_S fill:#cff,stroke:#333,stroke-width:2px
    style E_S fill:#cfc,stroke:#333,stroke-width:2px
```
-->

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/agent-decision-tree.png" alt="Agent Decision Tree" />

### Quick Reference Table

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based (sub_agents)** | Dynamic orchestration needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline → Write → Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |